# Project 1 - Data Cleaning Notebook

This jupyter notebook contains the cleaning of the diabetes_prediction_dataset on kaggle. The following code aims to tidy the dataset in a more usable format.

The first step is to import the dataframe from the csv, and save it to a new variable.Some of the columns in this dateset have 0 and 1 values, so these will be replace with True/False values.

In [2]:
import pandas as pd

df = pd.read_csv("Resources/diabetes_prediction_dataset.csv")

# creating a new variable for the clean dataframe
clean_df = df

# converting hypertension, heart_disease, and diabetes columns to boolean values
# people either have them, or they don't
clean_df["hypertension"] = clean_df["hypertension"].replace({0: False, 1: True})
clean_df["heart_disease"] = clean_df["heart_disease"].replace({0: False, 1: True})
clean_df["diabetes"] = clean_df["diabetes"].replace({0: False, 1: True})


clean_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,False,True,never,25.19,6.6,140,False
1,Female,54.0,False,False,No Info,27.32,6.6,80,False
2,Male,28.0,False,False,never,27.32,5.7,158,False
3,Female,36.0,False,False,current,23.45,5.0,155,False
4,Male,76.0,True,True,current,20.14,4.8,155,False
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,False,False,No Info,27.32,6.2,90,False
99996,Female,2.0,False,False,No Info,17.37,6.5,100,False
99997,Male,66.0,False,False,former,27.83,5.7,155,False
99998,Female,24.0,False,False,never,35.42,4.0,100,False


The relevant columns are now set to boolean values. It's time to take a closer look at the values for each column. First, we'll look at duplicate rows.

In [3]:
# look at duplicate rows
duplicates = clean_df[clean_df.duplicated()]

duplicates.count()

# duplicates.groupby("bmi").count()

# duplicates[duplicates["bmi"] == 27.32].describe()

gender                 3854
age                    3854
hypertension           3854
heart_disease          3854
smoking_history        3854
bmi                    3854
HbA1c_level            3854
blood_glucose_level    3854
diabetes               3854
dtype: int64

In [5]:
clean_df["smoking_history"].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

It is odd that 15 patients under the age of 5 are currently smoking, and that 61 patients under 5 have smoked in the past. Because this cannot be verified, we decided to limit the scope of our study to adults 21 and older. This will allow us to better determine the contributing factors to diabetes.

In [61]:
# limiting the scope of the study to adults over 21
clean_df = clean_df[clean_df["age"] > 21]

Let's deop the n/a values and check tbe values for all of the categorical and boolean columns.

In [22]:
# dropping na values
clean_df = clean_df.dropna()

list = ["gender", "hypertension", "heart_disease", "smoking_history", "diabetes"]
for x in list:
    print(clean_df[x].value_counts())

gender
Female    58552
Male      41430
Other        18
Name: count, dtype: int64
hypertension
False    92515
True      7485
Name: count, dtype: int64
heart_disease
False    96058
True      3942
Name: count, dtype: int64
smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64
diabetes
False    91500
True      8500
Name: count, dtype: int64


In the smoking_history column, some patients have been labeled as "ever". While this could be seen as a mistyping of "never", this cannot be confirmed. Therefore, these entries will be removed from the dataset.

In [54]:
# removing rows where smoking history is "ever"
clean_df = clean_df[clean_df["smoking_history"] != "ever"]

# checking remaining rows
clean_df.count()

gender                 95996
age                    95996
hypertension           95996
heart_disease          95996
smoking_history        95996
bmi                    95996
HbA1c_level            95996
blood_glucose_level    95996
diabetes               95996
dtype: int64

In [7]:
# viewing the current clean_df
clean_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,False,True,never,25.19,6.6,140,False
1,Female,54.0,False,False,No Info,27.32,6.6,80,False
2,Male,28.0,False,False,never,27.32,5.7,158,False
3,Female,36.0,False,False,current,23.45,5.0,155,False
4,Male,76.0,True,True,current,20.14,4.8,155,False
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,False,False,No Info,27.32,6.2,90,False
99996,Female,2.0,False,False,No Info,17.37,6.5,100,False
99997,Male,66.0,False,False,former,27.83,5.7,155,False
99998,Female,24.0,False,False,never,35.42,4.0,100,False


In [56]:
# creating a summary table comapring age, bmi, HbA1c levels, and blood glucose levels between people who have and don't have diabetes
summary_1 = clean_df.groupby("diabetes")[["age", "bmi", "HbA1c_level", "blood_glucose_level"]].agg(["mean", "median", "std"])

# transposing the dataframe so it is easier to read
summary_1.transpose()

diabetes                         False       True 
age                 mean     39.809312   61.023418
                    median   40.000000   62.000000
                    std      22.444480   14.601648
bmi                 mean     26.828584   31.995230
                    median   27.320000   29.950000
                    std       6.376896    7.580808
HbA1c_level         mean      5.396781    6.934716
                    median    5.800000    6.600000
                    std       0.971959    1.075827
blood_glucose_level mean    132.860597  194.387020
                    median  140.000000  160.000000
                    std      34.233557   58.713637

In [83]:
new_df = clean_df[clean_df["diabetes"] == True]

summary_2 = pd.Series(new_df.groupby(["heart_disease", "hypertension"])["diabetes"].count())


In [90]:
summary_2a = round(summary_2 / new_df["diabetes"].count(), 2)

summary_2a

heart_disease  hypertension
False          False           0.64
               True            0.21
True           False           0.11
               True            0.04
Name: diabetes, dtype: float64